In [1]:
import numpy as np
import pandas as pd

import pickle

from sklearn import naive_bayes
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier

from mlxtend.classifier import StackingClassifier

In [2]:
sf1_numbers = pd.read_csv('sf1_numbers_07.csv')
sf1_percentages = pd.read_csv('sf1_percentages_07.csv')

acs1_numbers = pd.read_csv('acs1_numbers_07.csv')
acs1_percentages = pd.read_csv('acs1_percentages_07.csv')

social_numbers = pd.read_csv('social_numbers_07.csv')
social_percentages = pd.read_csv('social_percentages_07.csv')

economic_numbers = pd.read_csv('economic_numbers_07.csv')
economic_percentages = pd.read_csv('economic_percentages_07.csv')

housing_numbers = pd.read_csv('housing_numbers_07.csv')
housing_percentages = pd.read_csv('housing_percentages_07.csv')

In [3]:
predictions = pd.read_csv('predictions.csv')

In [4]:
full_dfs = [sf1_numbers, sf1_percentages, acs1_numbers, acs1_percentages,
           social_numbers, social_percentages, economic_numbers,
           economic_percentages, housing_numbers, housing_percentages]

dfs = [sf1_numbers, acs1_numbers,social_numbers,economic_numbers,housing_numbers]

dfs_numbers = ['sf1_numbers', 'acs1_numbers', 'social_numbers', 'economic_numbers',
              'housing_numbers']

names = ['sf1_numbers', 'sf1_percentages', 'acs1_numbers',
         'acs1_percentages','social_numbers','social_percentages',
         'economic_numbers','economic_percentages','housing_numbers',
         'housing_percentages']

In [ ]:
for df in full_dfs:
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
    
predictions.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
for index, df in enumerate(full_dfs):
    df.name = names[index]

In [ ]:
y = predictions.loc[:,'Democrat']

scale_sf1 = StandardScaler()
scale_acs1 = StandardScaler()
scale_social = StandardScaler()
scale_economic = StandardScaler()
scale_housing = StandardScaler()

scalers = {'sf1_numbers': scale_sf1, 'acs1_numbers': scale_acs1, 'social_numbers': scale_social,
          'economic_numbers': scale_economic, 'housing_numbers': scale_housing}

kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)

models = {}

In [ ]:
nbg = naive_bayes.GaussianNB()
nbb = naive_bayes.BernoulliNB()
nbm = naive_bayes.MultinomialNB()

In [ ]:
for df in dfs:
    X = df.loc[:]
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 42)
    
    print(df.name,'\n')
    
    model1 = nbg.fit(X_train, y_train)
    
    print('Gaussian Naive Bayes')
    print("Cross Val Score:", np.mean(cross_val_score(model1,X_train,y_train,scoring='roc_auc',cv=kf)))
    print("AUC Score:", roc_auc_score(y_test, model1.predict(X_test)),'\n')
    
    #if roc_auc_score(y_test, model1.predict(X_test)) >= 0.75:
        #models[("{}".format(df.name), "Gaussian Naive Bayes")] = model1
    
    model2 = nbb.fit(X_train, y_train)
    
    print('Bernoulli Naive Bayes')
    print("Cross Val Score:", np.mean(cross_val_score(model2,X_train,y_train,scoring='roc_auc',cv=kf)))
    print("AUC Score:", roc_auc_score(y_test, model2.predict(X_test)),'\n')
    
    
    #if roc_auc_score(y_test, model1.predict(X_test)) >= 0.75:
        #models[("{}".format(df.name), "Bernoulli Naive Bayes")] = model2
    
    model3 = nbm.fit(X_train, y_train)
    
    print('Multinomial Naive Bayes')
    print("Cross Val Score:", np.mean(cross_val_score(model3,X_train,y_train,scoring='roc_auc',cv=kf)))
    print("AUC Score:", roc_auc_score(y_test, model3.predict(X_test)),'\n')
    
    #if roc_auc_score(y_test, model1.predict(X_test)) >= 0.75:
        #models[("{}".format(df.name), "Multinomial Naive Bayes")] = model3

sf1_numbers 

Gaussian Naive Bayes
Cross Val Score: 0.8883482321019077
AUC Score: 0.788562091503268 

Bernoulli Naive Bayes
Cross Val Score: 0.5
AUC Score: 0.5 

Multinomial Naive Bayes
Cross Val Score: 0.737861970681106
AUC Score: 0.7594771241830065 

acs1_numbers 

Gaussian Naive Bayes
Cross Val Score: 0.9146520232277998
AUC Score: 0.7895424836601308 

Bernoulli Naive Bayes
Cross Val Score: 0.5384004570321724
AUC Score: 0.5704248366013072 

Multinomial Naive Bayes
Cross Val Score: 0.8142354868273083
AUC Score: 0.8001633986928105 

social_numbers 

Gaussian Naive Bayes
Cross Val Score: 0.897576562493747
AUC Score: 0.7720588235294117 

Bernoulli Naive Bayes
Cross Val Score: 0.5
AUC Score: 0.5 

Multinomial Naive Bayes
Cross Val Score: 0.8019401127772909
AUC Score: 0.7839869281045752 

economic_numbers 

Gaussian Naive Bayes
Cross Val Score: 0.8944264574403797
AUC Score: 0.7859477124183006 

Bernoulli Naive Bayes
Cross Val Score: 0.5184906916713691
AUC Score: 0.5089869281045751 

Multin

In [ ]:
rf = RandomForestClassifier(n_estimators=10)

params = {'max_depth': range(1,21)}

grid = GridSearchCV(rf, params, scoring='roc_auc', cv=kf)

In [ ]:
for df in dfs:
    X = df.loc[:]
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 42)
    
    model = grid.fit(X_train, y_train)
    
    print(df.name, "Cross Val Score:", np.mean(cross_val_score(model, X_train, y_train, scoring='roc_auc',cv=kf)))
    print(df.name, "AUC Score:", roc_auc_score(y_test, model.predict(X_test)))
    print(df.name, "Best Depth:", model.best_params_, '\n')
    
    #if roc_auc_score(y_test, model.predict(X_test)) >= 0.75:
    models[("{}".format(df.name), "Random Forest")] = model.best_estimator_

sf1_numbers Cross Val Score: 0.9653831778543498
sf1_numbers AUC Score: 0.9161764705882353
sf1_numbers Best Depth: {'max_depth': 9} 



In [ ]:
dt = DecisionTreeClassifier()

params = {'max_depth': range(1,21)}

grid = GridSearchCV(dt, params, scoring='roc_auc', cv=kf)

In [ ]:
for df in dfs:
    X = df.loc[:]
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 42)
    
    model = grid.fit(X_train, y_train)
    
    print(df.name, "Cross Val Score:", np.mean(cross_val_score(model, X_train, y_train, scoring='roc_auc',cv=kf)))
    print(df.name, "AUC Score:", roc_auc_score(y_test, model.predict(X_test)))
    print(df.name, "Best Depth:", model.best_params_, '\n')
    
    #if roc_auc_score(y_test, model.predict(X_test)) >= 0.75:
    models[("{}".format(df.name), "Decision Tree")] = model.best_estimator_

In [ ]:
lvm = LinearSVC()

params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}

grid = GridSearchCV(lvm, params, scoring='roc_auc', cv=kf)

In [ ]:
for df in dfs:
    X = df.loc[:]
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 42)
    
    if df.name in dfs_numbers:
        X_train = scalers[df.name].fit_transform(X_train)
        X_test = scalers[df.name].transform(X_test)
    
    model = grid.fit(X_train, y_train)
    
    print(df.name, "Cross Val Score:", np.mean(cross_val_score(model, X_train, y_train, scoring='roc_auc',cv=kf)))
    print(df.name, "AUC Score:", roc_auc_score(y_test, model.predict(X_test)))
    print(df.name, "Best C:", model.best_params_, '\n')
    
    #if roc_auc_score(y_test, model.predict(X_test)) >= 0.75:
    models[("{}".format(df.name), "Linear SVM")] = model.best_estimator_

In [ ]:
svm = SVC(kernel = 'rbf', gamma = 'scale')

params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}

grid = GridSearchCV(svm, params, scoring='roc_auc', cv=kf)

In [ ]:
for df in dfs:
    X = df.loc[:]
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 42)
    
    if df.name in dfs_numbers:
        X_train = scalers[df.name].fit_transform(X_train)
        X_test = scalers[df.name].transform(X_test)
    
    model = grid.fit(X_train, y_train)
    
    print(df.name, "Cross Val Score:", np.mean(cross_val_score(model, X_train, y_train, scoring='roc_auc',cv=kf)))
    print(df.name, "AUC Score:", roc_auc_score(y_test, model.predict(X_test)))
    print(df.name, "Best C:", model.best_params_, '\n')
    
    #if roc_auc_score(y_test, model.predict(X_test)) >= 0.75:
    models[("{}".format(df.name), "RBF SVM")] = model.best_estimator_

In [ ]:
lr = LogisticRegression(solver='liblinear')

params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], 'penalty': ['l1','l2']}

grid = GridSearchCV(lr, params, scoring='roc_auc', cv=kf)

In [ ]:
for df in dfs:
    X = df.loc[:]
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 42)
    
    if df.name in dfs_numbers:
        X_train = scalers[df.name].fit_transform(X_train)
        X_test = scalers[df.name].transform(X_test)
    
    model = grid.fit(X_train, y_train)
    
    print(df.name, "Cross Val Score:", np.mean(cross_val_score(model, X_train, y_train, scoring='roc_auc',cv=kf)))
    print(df.name, "AUC Score:", roc_auc_score(y_test, model.predict(X_test)))
    print(df.name, "Best C And Penalty:", model.best_params_, '\n')
    
    #if roc_auc_score(y_test, model.predict(X_test)) >= 0.75:
    models[("{}".format(df.name), "Logistic Regression")] = model.best_estimator_

In [ ]:
knn = KNeighborsClassifier()

params = {'n_neighbors': range(1,11)}

grid = GridSearchCV(knn, params, scoring='roc_auc', cv=kf)

In [ ]:
for df in dfs:
    X = df.loc[:]
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 42)
    
    if df.name in dfs_numbers:
        X_train = scalers[df.name].fit_transform(X_train)
        X_test = scalers[df.name].transform(X_test)
    
    model = grid.fit(X_train, y_train)
    
    print(df.name, "Cross Val Score:", np.mean(cross_val_score(model, X_train, y_train, scoring='roc_auc',cv=kf)))
    print(df.name, "AUC Score:", roc_auc_score(y_test, model.predict(X_test)))
    print(df.name, "Best K:", model.best_params_, '\n')
    
    #if roc_auc_score(y_test, model.predict(X_test)) >= 0.75:
    models[("{}".format(df.name), "K Nearest Neighbor")] = model.best_estimator_

In [ ]:
predictions_sf1 = pd.DataFrame()
predictions_acs1 = pd.DataFrame()
predictions_social = pd.DataFrame()
predictions_economic = pd.DataFrame()
predictions_housing = pd.DataFrame()

prediction_dfs = [predictions_sf1, predictions_acs1, predictions_social, predictions_economic, predictions_housing]

for df in prediction_dfs:
    df['target'] = predictions['Democrat']

In [ ]:
models

In [ ]:
with open('models07.pickle', 'wb') as to_write:
    pickle.dump(models, to_write)

In [ ]:
for key, model in models.items():
    
    if key[0] == 'sf1_numbers':
        if key[1] in ['Linear SVM','RBF SVM','Logistic Regression','K Nearest Neighbor']:
            scaled = scalers['sf1_numbers'].fit_transform(sf1_numbers)
            predictions_sf1[key[1]] = model.predict(scaled)
        else:
            predictions_sf1[key[1]] = model.predict(sf1_numbers)
            
    elif key[0] == 'acs1_numbers':
        if key[1] in ['Linear SVM','RBF SVM','Logistic Regression','K Nearest Neighbor']:
            scaled = scalers['acs1_numbers'].fit_transform(acs1_numbers)
            predictions_acs1[key[1]] = model.predict(scaled)
        else:
            predictions_acs1[key[1]] = model.predict(acs1_numbers)
            
    elif key[0] == 'social_numbers':
        if key[1] in ['Linear SVM','RBF SVM','Logistic Regression','K Nearest Neighbor']:
            scaled = scalers['social_numbers'].fit_transform(social_numbers)
            predictions_social[key[1]] = model.predict(scaled)
        else:
            predictions_social[key[1]] = model.predict(social_numbers)
            
    elif key[0] == 'economic_numbers':
        if key[1] in ['Linear SVM','RBF SVM','Logistic Regression','K Nearest Neighbor']:
            scaled = scalers['economic_numbers'].fit_transform(economic_numbers)
            predictions_economic[key[1]] = model.predict(scaled)
        else:
            predictions_economic[key[1]] = model.predict(economic_numbers)
            
    elif key[0] == 'housing_numbers':
        if key[1] in ['Linear SVM','RBF SVM','Logistic Regression','K Nearest Neighbor']:
            scaled = scalers['housing_numbers'].fit_transform(housing_numbers)
            predictions_housing[key[1]] = model.predict(scaled)
        else:
            predictions_housing[key[1]] = model.predict(housing_numbers)
        

In [ ]:
predictions_sf1

In [ ]:
predictions_acs1.groupby(['target']).mean()

In [ ]:
predictions_social.groupby(['target']).mean()

In [ ]:
predictions_economic.groupby(['target']).mean()

In [ ]:
predictions_housing.groupby(['target']).mean()

In [ ]:
sf1_models = []
acs1_models = []
social_models = []
economic_models = []
housing_models = []

In [ ]:
for key, model in models.items():
    if key[0] == 'sf1_numbers': #and key[1] in ['Random Forest','RBF SVM','K Nearest Neighbor']:
        sf1_models.append((key[1],model))
    if key[0] == 'acs1_numbers': #and key[1] in ['Random Forest','RBF SVM','K Nearest Neighbor']:
        acs1_models.append((key[1],model))
    if key[0] == 'social_numbers': #and key[1] in ['Random Forest','RBF SVM','K Nearest Neighbor']:
        social_models.append((key[1],model))
    if key[0] == 'economic_numbers': #and key[1] in ['Random Forest','RBF SVM','K Nearest Neighbor']:
        economic_models.append((key[1],model))
    if key[0] == 'housing_numbers': #and key[1] in ['Random Forest','RBF SVM','K Nearest Neighbor']:
        housing_models.append((key[1],model))

In [ ]:
voting_sf1 = VotingClassifier(estimators=sf1_models, voting='hard',n_jobs=-1)
voting_acs1 = VotingClassifier(estimators=acs1_models, voting='hard',n_jobs=-1)
voting_social = VotingClassifier(estimators=social_models, voting='hard',n_jobs=-1)
voting_economic = VotingClassifier(estimators=economic_models, voting='hard',n_jobs=-1)
voting_housing = VotingClassifier(estimators=housing_models, voting='hard',n_jobs=-1)

In [ ]:
y = predictions['Democrat']
X = sf1_numbers

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 42)

voting_sf1.fit(X_train, y_train)
roc_auc_score(y_test, voting_sf1.predict(X_test))

In [ ]:
y = predictions['Democrat']
X = acs1_numbers

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 42)

voting_acs1.fit(X_train, y_train)
roc_auc_score(y_test, voting_acs1.predict(X_test))

In [ ]:
y = predictions['Democrat']
X = social_numbers

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 42)

voting_social.fit(X_train, y_train)
roc_auc_score(y_test, voting_social.predict(X_test))

In [ ]:
y = predictions['Democrat']
X = economic_numbers

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 42)

voting_economic.fit(X_train, y_train)
roc_auc_score(y_test, voting_economic.predict(X_test))

In [ ]:
y = predictions['Democrat']
X = economic_numbers

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 42)

voting_housing.fit(X_train, y_train)
roc_auc_score(y_test, voting_housing.predict(X_test))

In [ ]:
voters = [('voting_sf1', voting_sf1), ('voting_acs1', voting_acs1)]

voting = VotingClassifier(estimators = voters, voting='hard', n_jobs=-1)

